In [2]:
! unzip "/content/archive.zip" -d "/content/kaggle"

Archive:  /content/archive.zip
  inflating: /content/kaggle/Hindi_English_Truncated_Corpus.csv  


In [32]:
import numpy as np
import pandas as pd
import os
import  re
import string
from string import digits
from numpy import array, argmax, random, take
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, RepeatVector, TimeDistributed,Masking
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [61]:
df=pd.read_csv("/content/kaggle/Hindi_English_Truncated_Corpus.csv")
df=df[:10]

In [62]:
df.describe()

,source,english_sentence,hindi_sentence
count,10,10,10
unique,3,10,10
top,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
freq,4,1,1


In [63]:
df.dtypes

source              object
english_sentence    object
hindi_sentence      object
dtype: object

In [64]:
df['english_sentence'] = df['english_sentence'].astype(str)
df['english_sentence']=df['english_sentence'].apply(lambda x: x.lower())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.lower())

In [65]:
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub("'", '', x))
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation)
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['english_sentence']=df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [66]:
remove_digits = str.maketrans('', '', digits)
df['english_sentence']=df['english_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

df['english_sentence']=df['english_sentence'].apply(lambda x: x.strip())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

df['hindi_sentence']=df['hindi_sentence'].apply(lambda x:'START_ ' + x + ' _END')

In [67]:
senlen1=[]
for i in df['english_sentence']:
    res = len(i.split())
    senlen1.append(res)

df['engsentlen']=senlen1

In [68]:
senlen2=[]
for j in df['hindi_sentence']:
     res = len(j.split())
     senlen2.append(res)

df['hinsenlen']=senlen2

In [69]:
new_df=df[df['engsentlen']<=3]
new_df=df[df['hinsenlen']<=3]

max_length_eng=max(df['engsentlen'])
max_length_hin=max(df['hinsenlen'])

In [70]:
def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [71]:
english_tokenizer = tokenization(df['english_sentence'].values)

In [72]:
english_words=set()
for eng in df['english_sentence']:
    for word in eng.split():
        if word not in english_words:
            english_words.add(word)

hindi_words=set()
for hin in df['hindi_sentence']:
    for word in hin.split():
        if word not in hindi_words:
            hindi_words.add(word)

english_words.add(' ')
hindi_words.add(' ')

In [73]:
input_texts = []
target_texts = []

In [74]:
for i in df['english_sentence']:
    input_texts.append(i)

for i in df['hindi_sentence']:
    target_texts.append(i)

In [75]:
num_encoder_tokens = len(english_words)
num_decoder_tokens = len(hindi_words)

In [76]:
input_words = sorted(list(english_words))
target_words = sorted(list(hindi_words))

input_token_index = dict([(char, i) for i, char in enumerate(input_words)])
target_token_index = dict([(char, i) for i, char in enumerate(target_words)])

In [77]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [78]:
encoder_input_data = np.zeros((len(input_texts), max_length_eng, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_length_hin, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts),  max_length_hin, num_decoder_tokens),dtype='float32')

In [79]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t, input_token_index[word]]=1.

        encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
        for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t,target_token_index[word]]=1.
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.

        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
        decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [80]:
encoder_inputs=Input(shape=(None, num_encoder_tokens))
encoder=LSTM(128,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states=[state_h,state_c]

In [81]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [82]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [83]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, None, 100)]          0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, None, 121)]          0         []                            
                                                                                                  
 lstm_4 (LSTM)               [(None, 128),                117248    ['input_9[0][0]']             
                              (None, 128),                                                        
                              (None, 128)]                                                        
                                                                                            

In [84]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=128,epochs=20,validation_split=0.2)

Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 4.8244 - accuracy: 0.0034 - val_loss: 4.7908 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 44ms/step - loss: 4.6900 - accuracy: 0.0405 - val_loss: 4.7829 - val_accuracy: 0.0946
Epoch 3/20
1/1 [==============================] - 0s 45ms/step - loss: 4.5625 - accuracy: 0.6554 - val_loss: 4.7712 - val_accuracy: 0.0946
Epoch 4/20
1/1 [==============================] - 0s 46ms/step - loss: 4.3729 - accuracy: 0.6554 - val_loss: 4.7379 - val_accuracy: 0.0946
Epoch 5/20
1/1 [==============================] - 0s 40ms/step - loss: 3.9746 - accuracy: 0.6554 - val_loss: 4.5727 - val_accuracy: 0.0946
Epoch 6/20
1/1 [==============================] - 0s 42ms/step - loss: 2.9735 - accuracy: 0.6520 - val_loss: 4.8537 - val_accuracy: 0.0946
Epoch 7/20
1/1 [==============================] - 0s 41ms/step - loss: 2.0880 - accuracy: 0.6520 - val_loss: 5.1687 - val_accuracy: 0.0946
Epoch 8/20
1/1 [=========

In [85]:
model.save('final_model')

In [86]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [87]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['START_']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence = decoded_sentence + ' '+ sampled_char

        if (sampled_char == '_END' or
           len(decoded_sentence) > max_length_hin):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [88]:
for seq_index in range(6,9):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence in English:', input_texts[seq_index])
    print('Translated sentence in Hindi:', decoded_sentence)

1/1 [==============================] - 0s 21ms/step
-
Input sentence in English: in this lies the circumstances of people before you
Translated sentence in Hindi:  हम हम                                
1/1 [==============================] - 0s 20ms/step
-
Input sentence in English: and who are we to say even that they are wrong
Translated sentence in Hindi:  हम हम                                
1/1 [==============================] - 0s 21ms/step
-
Input sentence in English: “”global warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being
Translated sentence in Hindi:  हम यह                                
